In [1]:
import os
import shutil
from collections import OrderedDict
import json
import numpy as np

import matplotlib.pyplot as plt
import nibabel as nib

In [2]:
use_colab = False

if use_colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    sys.path.append('/content/drive/MyDrive/BrainMri')
    os.getcwd()
else:
    root_dir = os.getcwd()

my_nnunet_dir = os.path.join(root_dir, 'my_nnunet')
input_dir = os.path.join(root_dir, 'data/input')
segmentation_dir = os.path.join(root_dir, 'data/segmentation')
print(my_nnunet_dir)
print(input_dir)
print(segmentation_dir)

/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet
/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/data/input
/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/data/segmentation


In [3]:
def create_if_not_exist(folder_path, overwrite=False):
    """
    Create a folder if it does not exist.
    
    Args:
        folder_path (str): relative path of the folder which needs to be created
        overwrite (bool): Overwrite the existing folder if True. Defaults to False.
    """
    if os.path.exists(folder_path):
        if overwrite:
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)
            print(f'{folder_path} overwritten')
        else:
            print(f'{folder_path} already exists')
    else:
        os.makedirs(folder_path)
        print(f'{folder_path} created')

In [5]:
os.chdir(root_dir)
create_if_not_exist('my_nnunet', overwrite=False)
os.chdir('my_nnunet')
print(f'Current working directory: {os.getcwd()}')

my_nnunet already exists
Current working directory: /Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet


Now we create 2 folders where we can put our input images and corresponding segmentation

In [6]:
create_if_not_exist(input_dir)
create_if_not_exist(segmentation_dir)

/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/data/input created
/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/data/segmentation created


### Clone nnunet and install requirements

In [ ]:
!pip install nnunet

In [7]:
os.getcwd()

'/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet'

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
!git clone https://github.com/NVIDIA/apex

In [8]:
my_nnunet_dir

'/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet'

In [ ]:
repo_dir = os.path.join(my_nnunet_dir, 'nnUNet')
os.chdir(repo_dir)

In [ ]:
!pip install -e
!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

In [9]:
os.chdir(my_nnunet_dir)
os.getcwd()

'/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet'

### Create all required folders

In [11]:
task_name = 'Task001'

nnunet_dir = "nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
task_folder_name = os.path.join(nnunet_dir,task_name) 
train_image_dir = os.path.join(task_folder_name,'imagesTr') # path to training images
train_label_dir = os.path.join(task_folder_name,'labelsTr') # path to training labels
test_dir = os.path.join(task_folder_name,'imagesTs') # path to test images
main_dir = os.path.join(my_nnunet_dir,'nnUNet/nnunet') # path to main directory
trained_model_dir = os.path.join(main_dir, 'nnUNet_trained_models') # path to trained models

In [12]:
overwrite = False # Set this to True if you want to overwrite the folders
create_if_not_exist(task_folder_name,overwrite = overwrite)
create_if_not_exist(train_image_dir, overwrite = overwrite)
create_if_not_exist(train_label_dir, overwrite = overwrite)
create_if_not_exist(test_dir,overwrite= overwrite)
create_if_not_exist(trained_model_dir, overwrite=overwrite)

nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001 created
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTr created
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/labelsTr created
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTs created
/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/my_nnunet/nnUNet/nnunet/nnUNet_trained_models created


### Set env variables

In [ ]:
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = trained_model_dir

### Move files in training and test

In [13]:
def copy_rename(old_loc, old_fname, new_loc, new_fname, delete_old=False):
    """
    Copy a file from one location to another and rename it.
    
    Args:
        old_loc (str): Old location of the file.
        old_fname (str): Old name of the file.
        new_loc (str): New location of the file.
        new_fname (str): New name of the file.
        delete_old (bool): Delete the old file if True. Defaults to False.
    """
    old_file = os.path.join(old_loc, old_fname)
    new_file = os.path.join(new_loc, new_fname)
    shutil.copy(old_file, new_file)
    if delete_old:
        os.remove(old_file)

In [14]:
segmentation_dir

'/Users/toanne/Desktop/Brain-MRI-Segmentation/Brain-MRI-Segmentation/data/segmentation'

In [ ]:
list_of_all_files = os.listdir(segmentation_dir)
list_of_all_files = [fname for fname in list_of_all_files if fname.endswith('.nii.gz')]

for fname in list_of_all_files:
    copy_rename(input_dir, fname, train_image_dir, fname)
    copy_rename(segmentation_dir, fname, train_label_dir, fname)

In [15]:
def check_modality(fname):
    end = fname.find('.nii.gz')
    modality = fname[end-4:end]
    for mod in modality:
        if not(ord(mod) >= 48 and ord(mod) <= 57): # Check [0, 9]
            return False
    return True

In [ ]:
def rename_for_single_modality(dir):
    for file in os.listdir(dir):
        if check_modality(file):
            print(f'Modality present: {file}')
        else:
            new_name = file.replace('.nii.gz', '_0000.nii.gz')
            os.rename(os.path.join(dir, file), os.path.join(dir, new_name))
            print(f'Renamed to {new_name}')

rename_for_single_modality(train_image_dir)

### Set up JSON file

In [ ]:
overwrite_json_file = True # True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist == False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['name'] = task_name
    json_dict['description'] = "Segmentation of T1 Scans"
    json_dict['tensorImageSize'] = "3D"
    json_dict['reference'] = ""
    json_dict['licence'] = ""
    json_dict['release'] = ""

    # you may mention more than one modality
    json_dict['modality'] = {
        "0": "MRI"
    }

    # labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "0": "Non Brain",
        "1": "Cortical gray matter",
        "2": "Cortical White matter",
        "3" : "Cerebellum gray ",
        "4" : "Cerebellum white"
    }

    train_ids = os.listdir(train_label_dir)
    test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTest'] = len(test_ids)

    # no modality in train image and labels in dataset.json
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # remove the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % (i[:i.find("_0000")]+'.nii.gz') for i in test_ids]

    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else:
            print('dataset.json overwritten!')

### Preprocess the data for nnUNet format

In [ ]:
# -t 1 means "Task001", if you have a different task change it
!nnUNet_plan_and_preprocess -t 1 --verify_dataset_integrity

### Train Model

In [ ]:
# train 2D U net
!nnUNet_train 2d nnUNetTrainerV2 1 0 --npz

In [ ]:
# train 3D full resolution U net
!nnUNet_train 3d_fullres nnUNetTrainerV2 1 0 --npz 

In [ ]:
# train 3D U-net cascade
!nnUNet_train 3d_lowres nnUNetTrainerV2CascadeFullRes 1 0 --npz
!nnUNet_train 3d_fullres nnUNetTrainerV2CascadeFullRes 1 0 --npz

### Get the best config

!nnUNet_find_best_configuration -m 2d 3d_fullres 3d_lowres 3d_cascade_fullres -t 1

### Inference

In [ ]:
os.getcwd()

In [ ]:
task_folder_name

In [ ]:
result_dir = os.path.join(task_folder_name, 'nnUNet_Prediction_Results')
create_if_not_exist(result_dir, overwrite=True)

# -i is the input folder
# -o is where you want to save the predictions
# -t 1 means task 1, change it if you have a different task number
# Use -m 2d, or -m 3d_fullres, or -m 3d_cascade_fullres
!nnUNet_predict -i /content/drive/MyDrive/neurosciences-segmentation/my_nnunet/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTs -o /content/drive/MyDrive/neurosciences-segmentation/my_nnunet/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/nnUNet_Prediction_Results -t 1 -tr nnUNetTrainerV2 -m 2d -f 0  --num_threads_preprocessing 1